# Lab 3 — dimensionality reduction

In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np

from operator import itemgetter

%matplotlib inline
plt.style.use("ggplot")

In [5]:
!hdfs dfs -cat /ix/ml-20m/genome-tags.txt | tail -n 2

'hdfs' n'est pas reconnu en tant que commande interne
ou externe, un programme ex‚cutable ou un fichier de commandes.


In [6]:
data = sc.textFile("/ix/ml-20m/genome-tags.txt").map(json.loads)
tag2name = dict(data.map(itemgetter("tagId", "tag")).collect())

NameError: name 'sc' is not defined

In [8]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import HoverTool, ResetTool, PanTool, WheelZoomTool, SaveTool
output_notebook()

import pickle

Loading BokehJS ...

## Exercice 3.1

### Plot the variance of each dimension / tag.


### Plot the eigenvalues of the covariance matrix.


### Explain the implications for dimensionality reduction. For example, how many principal directions do you need to capture 2=3 of the variability in the data?

## Exercice 3.2

### For the five first principal directions, find the 10 tags that have the highest and lowest coordinates in that direction.

### What concepts would you use to describe these dimensions?

## Exercice 3.1

### Create an interactive plot that displays the 2D-projection of the movies using bokeh and its hover tool. Color the nodes by their Rotten Tomatoes score.

### Explain the coordinates of a few of the movies.

### How do the PCA directions correlate with the Rotten Tomatoes score?

## Exercice 3.1

### Plot the variance of each dimension / tag.


### Plot the eigenvalues of the covariance matrix.


### Explain the implications for dimensionality reduction. For example, how many principal directions do you need to capture 2=3 of the variability in the data?

## Exercice 3.2

### For the five first principal directions, find the 10 tags that have the highest and lowest coordinates in that direction.

### What concepts would you use to describe these dimensions?

## Exercice 3.3

### Create an interactive plot that displays the 2D-projection of the movies using bokeh and its hover tool. Color the nodes by their Rotten Tomatoes score.

In [30]:
# Reading an object from disk.
with open("selected-movies.pickle", "rb") as f:
    movies = pickle.load(f, encoding="utf-8")
    
print(len(movies)) 
print(movies['a'])

2
[0 1 2 3 4 5 6 7 8 9]


In [42]:
# Continuous colors
from matplotlib.colors import rgb2hex 
cmap = plt.get_cmap('viridis')
def val2rgb(val):
    return rgb2hex(cmap(val)[:3])

source = ColumnDataSource(
    data={
        "x": [x for x in movies['a']],
        "y": [x for x in movies['b']],
        "color":[val2rgb(x) for x in movies['a']]
    })

hover = HoverTool(
    tooltips=[
        ("Name", "@name"),
    ])
tools = [hover, ResetTool(), PanTool(), WheelZoomTool(), SaveTool()]

p = figure(plot_width=960, plot_height=360, tools=tools, title="Mouse over the dots")
p.circle("x", "y", source=source, size=20, color="color", alpha=0.5)
show(p)

### Explain the coordinates of a few of the movies.

### How do the PCA directions correlate with the Rotten Tomatoes score?